In [52]:
import os
import numpy as np
np.random.seed(43)

In [64]:
setup_string = '''
# General setup
open morph-open-closed/open-closed.pdb
center
turn y 60
scale 1.05

# Open bound structure get position of the ligand, missing in the morph
open 1ake
delete #1:.B
addh spec #1&protein hbond false
mmaker #0.1 #1

# Window appearance
windowsize 1920 1080
lighting reflectivity 0
set bg_color white
set silhouette
set silhouette_width 1.5

# Model appearance
color grey

# Hide everything
~ribbon; ~display
# Except ligand
display #1:AP5; color red #1:AP5; color byhet #1:AP5
'''

In [65]:
def pause_and_fade(pause_length, total_frames, file):
    file.write('# Fade out...\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format((pause + 1)* transparency_increment)
                  )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png supersample 3\n'.format(total_frames)
        )
    file.write('~display #1:AP5\n')
    return total_frames

In [66]:
def pause_and_fade_in(pause_length, total_frames, file):
    file.write('# Fade in...\n')
    file.write('display #1:AP5\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format(100 - ((pause + 1)* transparency_increment))
        )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png supersample 3\n'.format(total_frames)
        )

    return total_frames


def pause(pause_length, total_frames, file):
    for pause in np.arange(pause_length):
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png supersample 3 \n'.format(total_frames)
        )
    return total_frames

In [67]:
def add_bump(positive):
    sigma = 2
    if positive:
        mu = 6.0
    else:
        mu = -6.0
    rotation = sigma * np.random.randn(1) + mu
    # return rotation[0]
    return mu

In [68]:
def rotate_forward(models, residues, positives, 
                   list_of_previous_positions, file, colors, total_frames):
    #
    positions = np.empty(( len(residues), len(models) ))
    #
    for model in models:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues): 
            
            if model == models[0]:

                positions[residue_index, model] = \
                                list_of_previous_positions[residue_index, model] + \
                                add_bump(positives[residue_index])
            else:
                positions[residue_index, model] = \
                                positions[residue_index, model - 1] + \
                                add_bump(positives[residue_index])

            additional_rotation = positions[residue_index, model] - \
                                  list_of_previous_positions[residue_index, model]

            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


            print('Frame {:02d}: {:.2f} → {:.2f}, which is a rotation of additional {:.2f}'.format(model + 1, 
                                                               list_of_previous_positions[residue_index, model],
                                                               positions[residue_index, model],
                                                               additional_rotation))
            
        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png supersample 3\n\n'.format(total_frames)
        )

    return positions, total_frames

In [69]:
def rotate_backward(models, residues, positives, list_of_previous_positions, file, colors, total_frames):
    #
    positions = np.zeros(( len(residues), len(models) + 1))
    #
    for model in models[::-1]:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues):
            
            if model == models[-1]:

                positions[residue_index, model] = \
                                list_of_previous_positions[residue_index, model] + \
                                add_bump(positives[residue_index])
            else:
                positions[residue_index, model] = \
                                positions[residue_index, model + 1] + \
                                add_bump(positives[residue_index])
#                 print(f'{positions[residue_index, model]:.2f} = {positions[residue_index, model + 1]:.2f} + ' + \
#                       f'{add_bump(positives[residue_index]):.2f}')

            additional_rotation = positions[residue_index, model] - \
                                  list_of_previous_positions[residue_index, model]
            
            print('Frame {:02d}: {:.2f} → {:.2f}, which is a rotation of additional {:.2f}'.format(model + 1, 
                                                                               list_of_previous_positions[residue_index, model],
                                                                               positions[residue_index, model],
                                                                               additional_rotation))   
            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(additional_rotation))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')


        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png supersample 3\n\n'.format(total_frames)
        )
    
    return positions, total_frames

In [70]:
def gaussian(sigma, mu):
    rotation = sigma * np.random.randn(1) + mu
    return rotation[0]

def wiggle(frames, model, residue_list, total_frames, file):
    file.write('# Wiggle...\n')
    file.write('~display #1:AP5\n')

    file.write('ribbon #0.{}\n'.format(model))
    for frame in np.arange(frames):
        total_frames += 1
        for residue in residue_list:
            rotation = gaussian(10, -10)
            file.write('display #0.{}:{}\n'.format(model, residue))
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model, residue))
            file.write('rotation 2 {} 1; wait 1\n'.format(rotation))
            file.write('~rotation 2\n')

        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png supersample 3\n'.format(total_frames)
        )
    return total_frames

def add_text(string, frames, total_frames, file):
    file.write('2dlabel create title text "{}" xpos 0.1 ypos 0.92 color black\n'.format(string))
    file.write('2dlabel change title visibility hide frames {}\n'.format(frames))
    for frame in np.arange(frames):
        total_frames += 1
        file.write('wait 1\n')
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png supersample 3\n'.format(total_frames)
    )
    file.write('2dlabel delete title\n')
    return total_frames

In [71]:
residues = [20, 50, 120, 141, 200]
positives = [True, False, False, True, True]
pause_length = 20
morphs = np.arange(30)
total_frames = 0
colors = ['blue', 'green', 'purple', 'orange', 'yellow']

with open('morph_setup.cmd', 'w') as file:
    file.write(setup_string)
    # total_frames = wiggle(10, '1', wiggle_residues, total_frames, file)

    #total_frames = add_text(string='When a substrate binds a protein...',
    #                    frames=10,
    #                    total_frames=total_frames,
    #                    file=file
    #                   )


    file.write('2dlabel create title text "When a substrate binds a protein..." xpos 0.1 ypos 0.94 color black size 48\n')
    file.write('ribbon #0.1\n')
    total_frames = pause_and_fade_in(5, total_frames, file)

    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives,
        list_of_previous_positions=np.zeros(( len(residues), len(morphs) + 1)),
        file=file, 
        colors=colors,
        total_frames=total_frames)
    
    file.write('2dlabel change title text "...there is a conformational change." xpos 0.1 ypos 0.92 color black\n')
    
    total_frames = pause_and_fade(5, total_frames, file)
    print('')
        
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)

    
    total_frames = pause_and_fade_in(5, total_frames, file)
    
    print('')
    file.write('2dlabel change title text "Cycling between conformations..." xpos 0.1 ypos 0.92 color black\n')

    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives, 
        list_of_previous_positions=backward_rotations,
        file=file, 
        colors=colors,
        total_frames=total_frames)


    total_frames = pause_and_fade(5, total_frames, file)
    print('')
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)
    
    file.write('2dlabel change title text "...can lead to directional rotation of amino acid side chains." xpos 0.1 ypos 0.92 color black\n')

    total_frames = pause_and_fade_in(5, total_frames, file)
    print('')
    forward_rotations, total_frames = rotate_forward(
        models=morphs,
        residues=residues,
        positives=positives, 
        list_of_previous_positions=backward_rotations,
        file=file, 
        colors=colors,
        total_frames=total_frames)

    total_frames = pause_and_fade(5, total_frames, file)
    print('')
    backward_rotations, total_frames = rotate_backward(
        models=morphs, 
        residues=residues, 
        positives=positives, 
        list_of_previous_positions=forward_rotations,
        colors=colors,
        file=file, 
        total_frames=total_frames)

    total_frames = pause(5, total_frames, file)

Frame 01: 0.00 → 6.00, which is a rotation of additional 6.00
Frame 01: 0.00 → -6.00, which is a rotation of additional -6.00
Frame 01: 0.00 → -6.00, which is a rotation of additional -6.00
Frame 01: 0.00 → 6.00, which is a rotation of additional 6.00
Frame 01: 0.00 → 6.00, which is a rotation of additional 6.00
Frame 02: 0.00 → 12.00, which is a rotation of additional 12.00
Frame 02: 0.00 → -12.00, which is a rotation of additional -12.00
Frame 02: 0.00 → -12.00, which is a rotation of additional -12.00
Frame 02: 0.00 → 12.00, which is a rotation of additional 12.00
Frame 02: 0.00 → 12.00, which is a rotation of additional 12.00
Frame 03: 0.00 → 18.00, which is a rotation of additional 18.00
Frame 03: 0.00 → -18.00, which is a rotation of additional -18.00
Frame 03: 0.00 → -18.00, which is a rotation of additional -18.00
Frame 03: 0.00 → 18.00, which is a rotation of additional 18.00
Frame 03: 0.00 → 18.00, which is a rotation of additional 18.00
Frame 04: 0.00 → 24.00, which is a rot

```
ffmpeg -r 10 -i open-closed-%03d.png -y -vf format=yuv420p morph.mp4
```

Get sequence, add sequence index to `residues_list` if not GLY, PRO, or HIS.

In [61]:
fasta = 'MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELGKQAKDIMDAGKLVTDELVIALVKERIAQEDCRNGFLLDGFPRTIPQADAMKEAGINVDYVLEFDVPDELIVDRIVGRRVHAPSGRVYHVKFNPPKVEGKDDVTGEELTTRKDDQEETVRKRLVEYHQMTAPLIGYYSKEAEAGNTKYAKVDGTKPVAEVRADLEKILG'

In [62]:
wiggle_residues = []
for index, residue in enumerate(fasta, 1):
    if residue is not 'G' and residue is not 'P' and residue is not 'H':
        wiggle_residues.append(index)

For YouTube:
```
ffmpeg -i input.avi -c:v libx264 -preset slow -crf 18 -c:a copy -pix_fmt yuv420p output.mkv
```